In [6]:
import pandas as pd
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [27]:
# Load your dataset here (replace with your data loading code)
# Assuming you have a DataFrame 'data' with columns 'Filepath' and 'Label'
# Load your dataset here (replace with your data loading code)
file_path = "D:\Study Data\ICT Degree 7th sem\DL\dataset"
name_class = os.listdir(file_path)
filepaths = list(glob.glob(file_path+'/**/*.*'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))
filepath = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
data = pd.concat([filepath, labels], axis=1)
data = data.sample(frac=1).reset_index(drop=True)

# Split the dataset into training and testing sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Load the label encoder and fit it only on the training data
label_encoder = LabelEncoder()
label_encoder.fit(train['Label'])

# Transform the test labels using the fitted label encoder
test['Label'] = test['Label'].astype(str)

In [28]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data augmentation for testing data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_dataframe(
    train,
    x_col='Filepath',
    y_col='Label',  # Use the original 'Label' column with string labels
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_dataframe(
    test,
    x_col='Filepath',
    y_col='Label',  # Use the original 'Label' column with string labels
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)

# Encode the labels for training
label_encoder = LabelEncoder()
train_gen.classes = label_encoder.fit_transform(train_gen.classes)
test_gen.classes = label_encoder.transform(test_gen.classes)

Found 5545 validated image filenames belonging to 5 classes.
Found 1387 validated image filenames belonging to 5 classes.


In [29]:
# Define and compile the CNN model
cnn_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the CNN model
cnn_history = cnn_model.fit(
    train_gen,
    epochs=10,
    validation_data=test_gen,
    callbacks=[EarlyStopping(monitor='val_accuracy', min_delta=0, patience=2, mode='auto')]
)

Epoch 1/10
174/174 [==============================] - 216s 1s/step - loss: 1.0793 - accuracy: 0.5621 - val_loss: 0.6870 - val_accuracy: 0.7275
Epoch 2/10
174/174 [==============================] - 123s 707ms/step - loss: 0.7211 - accuracy: 0.7253 - val_loss: 0.5759 - val_accuracy: 0.8161
Epoch 3/10
174/174 [==============================] - 116s 665ms/step - loss: 0.6603 - accuracy: 0.7504 - val_loss: 0.5994 - val_accuracy: 0.7729
Epoch 4/10
174/174 [==============================] - 114s 655ms/step - loss: 0.6378 - accuracy: 0.7591 - val_loss: 0.5436 - val_accuracy: 0.8133


In [30]:
# Define and compile the ResNet model
resnet_model = ResNet50(
    input_shape=(100, 100, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

for layer in resnet_model.layers:
    layer.trainable = False

resnet_model.layers[-1].trainable = True

resnet_model = Sequential([resnet_model, Dense(len(label_encoder.classes_), activation='softmax')])

resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [31]:
# Train the ResNet model
resnet_history = resnet_model.fit(
    train_gen,
    epochs=10,
    validation_data=test_gen,
    callbacks=[EarlyStopping(monitor='val_accuracy', min_delta=0, patience=2, mode='auto')]
)

# Define and compile the VGG model
vgg_model = VGG16(
    input_shape=(100, 100, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

for layer in vgg_model.layers:
    layer.trainable = False

vgg_model.layers[-1].trainable = True

vgg_model = Sequential([vgg_model, Dense(len(label_encoder.classes_), activation='softmax')])

vgg_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Epoch 1/10
174/174 [==============================] - 138s 772ms/step - loss: 1.3913 - accuracy: 0.3814 - val_loss: 1.2521 - val_accuracy: 0.5198
Epoch 2/10
174/174 [==============================] - 131s 753ms/step - loss: 1.2456 - accuracy: 0.4734 - val_loss: 1.1825 - val_accuracy: 0.5133
Epoch 3/10
174/174 [==============================] - 133s 766ms/step - loss: 1.1981 - accuracy: 0.4949 - val_loss: 1.1059 - val_accuracy: 0.5689
Epoch 4/10
174/174 [==============================] - 136s 777ms/step - loss: 1.1586 - accuracy: 0.5169 - val_loss: 1.0814 - val_accuracy: 0.5292
Epoch 5/10
174/174 [==============================] - 135s 776ms/step - loss: 1.1470 - accuracy: 0.5253 - val_loss: 1.0334 - val_accuracy: 0.5804
Epoch 6/10
174/174 [==============================] - 135s 779ms/step - loss: 1.1150 - accuracy: 0.5412 - val_loss: 1.0211 - val_accuracy: 0.6063
Epoch 7/10
174/174 [==============================] - 141s 809ms/step - loss: 1.1029 - accuracy: 0.5544 - val_loss: 0.9932 -

In [32]:
# Train the VGG model
vgg_history = vgg_model.fit(
    train_gen,
    epochs=10,
    validation_data=test_gen,
    callbacks=[EarlyStopping(monitor='val_accuracy', min_delta=0, patience=2, mode='auto')]
)

Epoch 1/10
174/174 [==============================] - 184s 1s/step - loss: 1.2307 - accuracy: 0.5427 - val_loss: 1.0151 - val_accuracy: 0.6965
Epoch 2/10
174/174 [==============================] - 162s 930ms/step - loss: 0.9556 - accuracy: 0.7073 - val_loss: 0.8451 - val_accuracy: 0.7152
Epoch 3/10
174/174 [==============================] - 155s 889ms/step - loss: 0.8345 - accuracy: 0.7423 - val_loss: 0.7521 - val_accuracy: 0.7448
Epoch 4/10
174/174 [==============================] - 155s 892ms/step - loss: 0.7756 - accuracy: 0.7499 - val_loss: 0.7128 - val_accuracy: 0.7477
Epoch 5/10
174/174 [==============================] - 155s 891ms/step - loss: 0.7213 - accuracy: 0.7574 - val_loss: 0.6583 - val_accuracy: 0.7794
Epoch 6/10
174/174 [==============================] - 156s 894ms/step - loss: 0.6862 - accuracy: 0.7711 - val_loss: 0.6190 - val_accuracy: 0.8010
Epoch 7/10
174/174 [==============================] - 156s 897ms/step - loss: 0.6691 - accuracy: 0.7760 - val_loss: 0.6102 - va

In [33]:
# Compare and visualize results
cnn_test_results = cnn_model.evaluate(test_gen, verbose=0)
resnet_test_results = resnet_model.evaluate(test_gen, verbose=0)
vgg_test_results = vgg_model.evaluate(test_gen, verbose=0)

print("CNN Test Accuracy: {:.2f}%".format(cnn_test_results[1] * 100))
print("ResNet Test Accuracy: {:.2f}%".format(resnet_test_results[1] * 100))
print("VGG Test Accuracy: {:.2f}%".format(vgg_test_results[1] * 100))

CNN Test Accuracy: 81.33%
ResNet Test Accuracy: 66.91%
VGG Test Accuracy: 79.96%


In [34]:
# Choose the best model based on test accuracy
best_model = None
best_model_name = ""
if cnn_test_results[1] >= resnet_test_results[1] and cnn_test_results[1] >= vgg_test_results[1]:
    best_model = cnn_model
    best_model_name = "CNN"
elif resnet_test_results[1] >= vgg_test_results[1]:
    best_model = resnet_model
    best_model_name = "ResNet"
else:
    best_model = vgg_model
    best_model_name = "VGG"

# Print the best model's name
print(f"The best model is {best_model_name}")

# Evaluate and visualize the best model
best_test_results = best_model.evaluate(test_gen, verbose=0)
print("Best Model Test Accuracy: {:.2f}%".format(best_test_results[1] * 100))

The best model is CNN
Best Model Test Accuracy: 81.33%


In [ ]:
# # Predict the label of the test_gen using the best model
# best_pred = best_model.predict(test_gen)
# best_pred = np.argmax(best_pred, axis=1)

# # Map the label
# labels = label_encoder.classes_
# best_pred = [labels[k] for k in best_pred]

# y_test = list(test_gen.filenames)
# y_test = [label_encoder.transform([y])[0] for y in y_test]
# print(classification_report(y_test, best_pred))

In [ ]:
# # Visualize results for the best model
# fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(12, 8), subplot_kw={'xticks': [], 'yticks': []})
# for i, ax in enumerate(axes.flat):
#     ax.imshow(plt.imread(test_gen.filepaths[i]))
#     ax.set_title(f"True: {label_encoder.inverse_transform([y_test[i]])[0]}\nPredicted: {best_pred[i]}")
# plt.tight_layout()
# plt.show()